In [1]:
import pandas as pd
import numpy as np
import time
import re
from tqdm import tqdm_notebook
from torrequest import TorRequest
from bs4 import BeautifulSoup
from retrying import retry
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ECimport 
import pandas as pd
import souper_v2
from tqdm import tqdm_notebook

In [2]:
# Backup of original simplier function to get_soup
def get_soup(URL):
    
    chrome_options = Options()
    chrome_options.add_argument('--window-size=1,1')
    
    #options = webdriver.ChromeOptions()
    #options.add_argument('--disable-popup-blocking')
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)
    wait = WebDriverWait(driver, 10)
    driver.get(URL)
    if len(driver.find_elements_by_css_selector("[name=ROBOTS]")) > 0:
        driver.get(URL)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return soup

In [3]:
sampled_players = pd.read_csv('C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Obtaining_players_and_teams/players_transfermarket.csv')

In [4]:
sampled_players.head()

,Unnamed: 0,link,player,team
0,0,/david-de-gea/profil/spieler/59377,David de Gea,Manchester United
1,1,/jack-butland/profil/spieler/128899,Jack Butland,Manchester United
2,2,/tom-heaton/profil/spieler/34130,Tom Heaton,Manchester United
3,3,/lisandro-martinez/profil/spieler/480762,Lisandro Martínez,Manchester United
4,4,/raphael-varane/profil/spieler/164770,Raphaël Varane,Manchester United


In [5]:
def extract_injuries(soup):
    injuries = []
    for each in soup.find_all('td', attrs={'class':'rechts'}):
        try:
            injuries.append(each.text)
        except:
            None
    
    return injuries

In [6]:
# Thu thập thông tin về lịch sử chấn thương của từng cầu thủ
def get_injuries(players_df):
    
    links = players_df['link']
    
    injuries= {'player':[], 'days_injured': [], 'games_injured': []}

    for each_link in links:
        
        # Get injuries data
        results = 0
        new_link = each_link.replace("profil","verletzungen")
        new_link = 'https://www.transfermarkt.co.uk' + new_link
        print(new_link)
        soup = get_soup(new_link)
        results = extract_injuries(soup)
        
        try:
            days_injured = sum([int(word.replace(" days","")) for word in [x for x in results if results.index(x)%2 == 0]])
            games_injured = sum([int(word.replace('-', "0")) for word in [x for x in results if results.index(x)%2 != 0]])
        except:
            days_injured = 0
            games_injured = 0
        
        injuries['days_injured'].append(days_injured)
        injuries['games_injured'].append(games_injured)
        injuries['player'].append(each_link)
        
        element_index = links[links == each_link].index[0]
        print(element_index)
            
    return injuries

In [7]:
# Sample_data là một list, mỗi phần tử là 1 dataframe tách ra từ sampled_players
sample_data = []
sample_data.append(sampled_players.iloc[:10])
i = 10
while i < sampled_players.shape[0]:
    if i >= sampled_players.shape[0] - 200:
        sample_data.append(sampled_players.iloc[i:])
    else:
        sample_data.append(sampled_players.iloc[i:i+200])
    i += 200 # mỗi sample chứa 200 dòng
print(len(sample_data))

55


In [32]:
print(sample_data[48])

      Unnamed: 0                                      link             player  \
9410           3    /amin-al-bukhari/profil/spieler/509875    Amin Al-Bukhari   
9411           4     /waleed-abdullah/profil/spieler/86282    Waleed Abdullah   
9412           5    /alvaro-gonzalez/profil/spieler/183647    Álvaro González   
9413           6  /abdulelah-al-amri/profil/spieler/509894  Abdulelah Al-Amri   
9414           7      /abdullah-madu/profil/spieler/265940      Abdullah Madu   
...          ...                                       ...                ...   
9605          18       /seo-joon-bae/profil/spieler/946989       Seo-joon Bae   
9606          19      /jong-hyun-lee/profil/spieler/754801      Jong-hyun Lee   
9607          20         /jae-suk-oh/profil/spieler/126642         Jae-suk Oh   
9608          21         /ji-hun-kim/profil/spieler/643383         Ji-hun Kim   
9609          22        /seon-yu-lee/profil/spieler/821666        Seon-yu Lee   

                      team 

In [14]:
# Muốn cào sample nào thì thay i bàng sample đó
i = 48
sample = get_injuries(sample_data[i])

https://www.transfermarkt.co.uk/amin-al-bukhari/verletzungen/spieler/509875
9410
https://www.transfermarkt.co.uk/waleed-abdullah/verletzungen/spieler/86282
9411
https://www.transfermarkt.co.uk/alvaro-gonzalez/verletzungen/spieler/183647
9412
https://www.transfermarkt.co.uk/abdulelah-al-amri/verletzungen/spieler/509894
9413
https://www.transfermarkt.co.uk/abdullah-madu/verletzungen/spieler/265940
9414
https://www.transfermarkt.co.uk/ali-al-oujami/verletzungen/spieler/264909
9415
https://www.transfermarkt.co.uk/mohammed-al-fatil/verletzungen/spieler/193150
9416
https://www.transfermarkt.co.uk/yousef-haqawi/verletzungen/spieler/935927
9417
https://www.transfermarkt.co.uk/ghislain-konan/verletzungen/spieler/422850
9418
https://www.transfermarkt.co.uk/majed-qasheesh/verletzungen/spieler/1065823
9419
https://www.transfermarkt.co.uk/sultan-al-ghannam/verletzungen/spieler/435045
9420
https://www.transfermarkt.co.uk/nawaf-al-boushal/verletzungen/spieler/821377
9421
https://www.transfermarkt.co.

9514
https://www.transfermarkt.co.uk/han-jin-kwon/verletzungen/spieler/195550
9515
https://www.transfermarkt.co.uk/dae-jung-kim/verletzungen/spieler/379073
9516
https://www.transfermarkt.co.uk/hyung-jin-lim/verletzungen/spieler/1087019
9517
https://www.transfermarkt.co.uk/gun-hee-kim/verletzungen/spieler/1060671
9518
https://www.transfermarkt.co.uk/dong-yun-jeong/verletzungen/spieler/428654
9519
https://www.transfermarkt.co.uk/myung-joo-lee/verletzungen/spieler/217569
9520
https://www.transfermarkt.co.uk/do-hyeok-kim/verletzungen/spieler/319363
9521
https://www.transfermarkt.co.uk/reum-yeo/verletzungen/spieler/379410
9522
https://www.transfermarkt.co.uk/dong-su-lee/verletzungen/spieler/497374
9523
https://www.transfermarkt.co.uk/hyun-bin-park/verletzungen/spieler/743779
9524
https://www.transfermarkt.co.uk/jin-hong-park/verletzungen/spieler/1087018
9525
https://www.transfermarkt.co.uk/jun-yub-kim/verletzungen/spieler/173269
9526
https://www.transfermarkt.co.uk/yun-gu-gang/verletzungen/

In [15]:
print(pd.DataFrame(sample))

                                       player  days_injured  games_injured
0      /amin-al-bukhari/profil/spieler/509875             0              0
1       /waleed-abdullah/profil/spieler/86282           225             16
2      /alvaro-gonzalez/profil/spieler/183647            79             13
3    /abdulelah-al-amri/profil/spieler/509894             0              0
4        /abdullah-madu/profil/spieler/265940             0              0
..                                        ...           ...            ...
195       /seo-joon-bae/profil/spieler/946989             0              0
196      /jong-hyun-lee/profil/spieler/754801            74             15
197         /jae-suk-oh/profil/spieler/126642           338             53
198         /ji-hun-kim/profil/spieler/643383             0              0
199        /seon-yu-lee/profil/spieler/821666             0              0

[200 rows x 3 columns]


In [16]:
sample = pd.DataFrame(sample)
file_name = "C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Injuries_player/sample_data_" + str(i) + ".csv"
sample.to_csv(file_name) # Lưu thành file sample_data_i.csv